In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
import pandas as pd
import tensorflow_text as tf_text

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display
from pydub import AudioSegment

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
#asignamos el tamaño de las etiquetas y el dataframe
infodataset = pd.read_csv("asr_train2.tsv", sep="\t")
infodataset['id']=infodataset['id'].astype(str)
max_vocab_size = 10000


In [ ]:
#Hacemos procesos inicales para el dataset
Dataseth_path = "./dataset"
data_dir = pathlib.Path(Dataseth_path)
archivosAudio = tf.io.gfile.glob(str(data_dir)+ '/*')
archivosAudio = tf.random.shuffle(archivosAudio)
num_audios = len(archivosAudio)
print(num_audios)
print(archivosAudio[0])

In [ ]:
def codec16k(audio_original):
    audioAux = AudioSegment.from_file(audio_original)
    audioAux = audioAux.set_frame_rate(16000)
    nombreAudio = os.path.basename(os.path.abspath(audio_original))
    os.remove(audio_original)
    audioFin = audioAux.export(nombreAudio,format="wav",codec="pcm_s16le")
    return audioFin


In [ ]:
def decode_audio(audio_binary):
    audio, _ = tf.audio.decode_wav(contents=audio_binary)
    return tf.squeeze(audio, axis= -1)

In [ ]:
def vectorizarEtiqueta(texto):
    texto = texto.replace('.','')
    tensorTexto = tf.strings.split(input = texto, sep=' ')
    return tensorTexto


In [ ]:
def buscar_etiqueta(audio_id):
    etiqueta = infodataset.loc[infodataset['id']==audio_id,'normalized_text']
    return etiqueta.iloc[0]

In [ ]:
def waverformConEtiquetas(Audio):
    etiqueta_sacada = str(buscar_etiqueta(Audio))
    etiqueta = vectorizarEtiqueta(etiqueta_sacada)
    AudioNormalizado = codec16k(Audio)
    audio_binary = tf.io.read_file(AudioNormalizado)
    AudioProcesado = decode_audio(audio_binary)
    return AudioProcesado , etiqueta

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
Archivos_ds = tf.data.Dataset.from_tensor_slices(archivosAudio)
waverform_ds = Archivos_ds.map(
    map_func=waverformConEtiquetas,
    num_parallel_calls=AUTOTUNE
)


In [ ]:
print(waverformConEtiquetas("ejemplo2.wav"))

In [ ]:
print(infodataset.to_string())